In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import os
import time
import pyproj
import glob
import pickle

In [3]:
airport_location = pd.read_csv('../data/airport_positions.csv')

In [4]:
# Remove parenthesis for airport location in dataset -> split lon and lat values -> divide by 100 to obtain correct values
airport_location[['latitude', 'longitude']] = airport_location['position'].str.replace(r'[\(\)]', '').str.split(',', expand=True)
airport_location[['latitude', 'longitude']] = airport_location[['latitude', 'longitude']].astype('float') / 100

/tmp/ipykernel_1887808/664398462.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  airport_location[['latitude', 'longitude']] = airport_location['position'].str.replace(r'[\(\)]', '').str.split(',', expand=True)


In [5]:
parameters = ['air_temperature_0m', 'air_temperature_2m', 'air_temperature_pl', 
              'relative_humidity_2m', 'precipitation_amount_acc', 'x_wind_10m', 'y_wind_10m', 
             'x_wind_pl', 'y_wind_pl', 'fog_area_fraction', 'surface_air_pressure', 'air_pressure_at_sea_level']

In [7]:
# Define folder with paths and prefix for files to read
folder_path = '/lustre/storeB/immutable/archive/projects/metproduction/meps/2022/10/'
file_prefix = 'meps_lagged_6_h_subset_2_5km_'
file_pattern = os.path.join(folder_path, file_prefix)
file_list = glob.glob(file_pattern)
output_folder = '/lustre/storeB/users/tonjek/msc/2024_msc_tonje_metar/data_extraction/2022/'

In [8]:
def transform_latitude_longitude_to_xy(latitude_values, longitude_values):
    # Create a pyproj CRS object
    proj4_str = '+proj=lcc +lat_0=63.3 +lon_0=15 +lat_1=63.3 +lat_2=63.3 +no_defs +R=6.371e+06'
    lcc_crs = pyproj.CRS.from_proj4(proj4_str)

    # Create a transformer for converting between lat/lon and x/y
    transformer = pyproj.Transformer.from_crs(pyproj.CRS("EPSG:4326"), lcc_crs, always_xy=True)

    # Transform lat/lon to x/y
    x_values, y_values = transformer.transform(longitude_values, latitude_values)

    return x_values, y_values

In [25]:
all_timesteps_data = []
for day_folder in os.listdir(folder_path):
        day_path = os.path.join(folder_path, day_folder)

        file_list = [file for file in os.listdir(day_path) if file.startswith(file_prefix)]

        for file in file_list:
            file_path = os.path.join(day_path, file)

            start_time = time.time()

            data = xr.open_mfdataset(file_path, chunks={'time':1})
            print(f"Processing file: {file_path}")

            # Extracting parameter values only for matching nearest location
            latitude_values = airport_location.latitude.values
            longitude_values = airport_location.longitude.values
            
            x_values, y_values = transform_latitude_longitude_to_xy(latitude_values, longitude_values)
            
            airport_six_timesteps = []

            for airport_idx in range(len(latitude_values)):
                nearest_x, nearest_y = transform_latitude_longitude_to_xy(latitude_values[airport_idx], longitude_values[airport_idx])
                
                lat_from_data = data.latitude.sel(x=nearest_x, y=nearest_y, method='nearest').values.item()
                lon_from_data = data.longitude.sel(x=nearest_x, y=nearest_y, method='nearest').values.item()


                interpolated_data = data[parameters].interp(x=nearest_x, y=nearest_y)

                # Convert to pandas datetime
                start_datetime = pd.to_datetime(data.time.values[0])
                # Choosing 6 timesteps in total
                end_datetime = start_datetime + pd.DateOffset(hours=5)
                

                extracted_data = interpolated_data.sel(
                    ensemble_member=0,
                    time=slice(start_datetime, end_datetime)
                )
                
                extracted_data = extracted_data.expand_dims({'airport': [airport_idx + 1]})

                airport_six_timesteps.append(extracted_data)
        combined_data = xr.concat(airport_six_timesteps, dim='airport')
        all_timesteps_data.append(combined_data)
        end_time = time.time()
        print(f'Time taken to extract info from {file}: {end_time - start_time}')
        print(len(all_timesteps_data))
final_combined_data = xr.concat(all_timesteps_data, dim='time')


Processing file: /lustre/storeB/immutable/archive/projects/metproduction/meps/2021/10/19/meps_lagged_6_h_subset_2_5km_20211019T12Z.nc
Processing file: /lustre/storeB/immutable/archive/projects/metproduction/meps/2021/10/19/meps_lagged_6_h_subset_2_5km_20211019T18Z.nc
Processing file: /lustre/storeB/immutable/archive/projects/metproduction/meps/2021/10/19/meps_lagged_6_h_subset_2_5km_20211019T00Z.nc
Processing file: /lustre/storeB/immutable/archive/projects/metproduction/meps/2021/10/19/meps_lagged_6_h_subset_2_5km_20211019T06Z.nc
Time taken to extract info from meps_lagged_6_h_subset_2_5km_20211019T06Z.nc: 2.781939744949341
1
Processing file: /lustre/storeB/immutable/archive/projects/metproduction/meps/2021/10/11/meps_lagged_6_h_subset_2_5km_20211011T00Z.nc
Processing file: /lustre/storeB/immutable/archive/projects/metproduction/meps/2021/10/11/meps_lagged_6_h_subset_2_5km_20211011T12Z.nc
Processing file: /lustre/storeB/immutable/archive/projects/metproduction/meps/2021/10/11/meps_lagg

In [ ]:
pickle_filename = f'2022_10_test.pkl'
pickle_filepath = os.path.join(output_folder, pickle_filename)

with open(pickle_filepath, 'wb') as pickle_file:
    pickle.dump(final_combined_data, pickle_file)

print(f"Saved pickle file for file {file_path}: {pickle_filepath}")